In [1]:
# ResNet18 by pytorch - with pretrain
# ColorJitter-contrast every dataset
# Use fc(512, 128, 32, 5) 
# With Batch Normalization
# With freezen
# epochs : 50

# optim : Adam

# test_acc : 
# val_acc : 
# model_path : ResNet18v2.pth

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import os

In [3]:
TEST_DATA_PATH =  "./test"
TRAIN_DATA_PATH = "./train"

In [5]:
# data transform, you can add different transform methods
img_size = 224
torch.manual_seed(0)

train_orig = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_aug0 = transforms.Compose([
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_aug1 = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.RandomAffine(0,scale=(0.7,1.2)),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_aug2 = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.transforms.RandomRotation(25),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_aug3 = transforms.Compose([
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

train_aug5 = transforms.Compose([
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomVerticalFlip(),
                                    transforms.ColorJitter(contrast = 0.5),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])



test_transform = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
                                ])

dataset_orig = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_orig)
dataset_aug0 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug0)
dataset_aug1 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug1)
dataset_aug2 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug2)
dataset_aug3 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug3)
dataset_aug5 = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_aug5)


dataset_list = [dataset_orig, dataset_aug0, dataset_aug1, dataset_aug2, dataset_aug3, dataset_aug5]

dataset = data.ConcatDataset(dataset_list)


test_data = datasets.ImageFolder(root=TEST_DATA_PATH,transform=test_transform)

# spilt data into training and validation
TOTAL_SIZE = len(dataset)
ratio = 0.9
train_len = round(TOTAL_SIZE * ratio)
val_len = round(TOTAL_SIZE * (1-ratio))

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, val_len])

# data loader, you can choose the input arguments by yourself

train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=128, shuffle=True,  num_workers=4)
test_data_loader  = data.DataLoader(test_data, batch_size=8, shuffle=False, num_workers=4) 

print(dataset)
#print(dataset.class_to_idx)

In [6]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
print(len(train_data_loader.dataset))

8888


In [8]:
model = torchvision.models.resnet18(pretrained=True)
num = model.fc.in_features
model.fc = nn.Linear(num, 128)
model = nn.Sequential(model, nn.BatchNorm1d(128))
model = nn.Sequential(model, nn.Linear(128, 32))
model = nn.Sequential(model, nn.BatchNorm1d(32))
model = nn.Sequential(model, nn.Linear(32, 5))

print(model)
model.to(device=device)

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): ResNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (relu): ReLU(inplace)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (1): BasicBlock(
              (conv1): Conv2d(64, 64, k

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Sequential(
        (0): ResNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): BasicBlock(
              (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (relu): ReLU(inplace)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (1): BasicBlock(
              (conv1): Conv2d(64, 64, k

In [9]:
def set_grad(model, is_extract):
    if is_extract:
        for param in list(model.parameters())[:-4]:
            param.requires_grad = False
#set_grad(model, True)
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
#         print("\t",name)

learning_rate=1e-2

'''optimizer = optim.Adam(model.parameters(), lr=0.001)
lr_sch = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
criterion = nn.CrossEntropyLoss()'''
optimizer = optim.Adam(params_to_update, lr=0.001)
#lr_sch = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
criterion = nn.CrossEntropyLoss()

# start training 
epochs = 50

min_val_loss = float("inf")

for epoch in range(1, epochs+1):
    model.train()
    total_train_loss = 0
    total_val_loss = 0
    train_hit = 0
    val_hit = 0
    print(f"{epoch}  {optimizer.param_groups[0]['lr']}")

    
    
    for data, target in train_data_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()

        output=model(data)
        
        # loss function
        loss = criterion(output, target)

        total_train_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        train_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 


        # do back propagation
        loss.backward()
        optimizer.step()
    
    
    with torch.no_grad():
        model.eval()
        for data, target in val_data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            total_val_loss += F.cross_entropy(output, target).item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            val_hit += pred.eq(target.data.view_as(pred)).cpu().sum().item() 
    
    avg_train_loss = total_train_loss/len(train_data_loader)
    avg_val_loss   = total_val_loss/len(val_data_loader)
    
    print('Epoch:%3d'%epoch
        , '|Train Loss:%8.4f'%(avg_train_loss)
        , '|Train Acc:%3.4f'%(train_hit/len(train_data_loader.dataset)*100.0)
        , '|Val Loss:%8.4f'%(avg_val_loss)
        , '|Val Acc:%3.4f'%(val_hit/len(val_data_loader.dataset)*100.0))
    
    if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "ResNet18v2.pth")
    

1  0.001
Epoch:  1 |Train Loss:  0.8834 |Train Acc:71.3209 |Val Loss:  1.0329 |Val Acc:60.1215
-------------saving model--------------
2  0.001
Epoch:  2 |Train Loss:  0.6074 |Train Acc:79.2529 |Val Loss:  0.9688 |Val Acc:66.1943
-------------saving model--------------
3  0.001
Epoch:  3 |Train Loss:  0.5258 |Train Acc:80.4793 |Val Loss:  0.6177 |Val Acc:74.5951
-------------saving model--------------
4  0.001
Epoch:  4 |Train Loss:  0.4440 |Train Acc:83.2696 |Val Loss:  0.6002 |Val Acc:77.3279
-------------saving model--------------
5  0.001
Epoch:  5 |Train Loss:  0.3813 |Train Acc:85.6436 |Val Loss:  0.7131 |Val Acc:75.4049
6  0.001
Epoch:  6 |Train Loss:  0.3547 |Train Acc:86.5549 |Val Loss:  0.4714 |Val Acc:82.0850
-------------saving model--------------
7  0.001
Epoch:  7 |Train Loss:  0.3243 |Train Acc:87.8038 |Val Loss:  0.5561 |Val Acc:78.9474
8  0.001
Epoch:  8 |Train Loss:  0.3171 |Train Acc:87.7700 |Val Loss:  0.3557 |Val Acc:85.5263
-------------saving model--------------


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model so that you don't need to train the model again
test_model = torch.load("ResNet18v2.pth").to(device)

In [14]:
def test(model,data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = test_data.samples[i*bs+j][0].split('/')[-1]
                result.append((file_name,pred[j].cpu().numpy()[0]))
    return result

In [15]:
result = test(test_model,test_data_loader)

In [16]:
with open ('ID_result_ad_18_diflr.csv','w') as f:
    f.write('ID,label\n')
    for data in result:
        f.write(data[0]+','+str(data[1])+'\n')